# Generate a single [character.ai](https://beta.character.ai/) character definition

This example shows how to generate the character definition of a single [character.ai](https://beta.character.ai/) character from a corpus. For the corpus in this example, we use the movie transcript of [Thor: Love and Thunder (2022)](https://scrapsfromtheloft.com/movies/thor-love-and-thunder-transcript/).

To generate your own character definition:
1. Put the corpus into a single a `.txt` file inside the `data/` directory.
2. Assign the name of the `.txt` file to the `CORPUS` constant below.
3. Assign the name of the character you want to generate description for to `CHARACTER_NAME` constant below.
4. Run this notebook.

In [1]:
CORPUS = 'data/thor_love_and_thunder.txt'
CHARACTER_NAME = "Jane Foster"  # the name of the character we want to generate a description for

In [2]:
from dataclasses import asdict
import json
import os

from src.character import get_character_definition
from src.corpus import get_characters, get_rolling_summaries, load_docs

In [3]:
# create directories to cache results and intermediate outputs
OUTPUT_ROOT = "output"
corpus_name = os.path.splitext(os.path.basename(CORPUS))[0]
output_dir = f"{OUTPUT_ROOT}/{corpus_name}"
os.makedirs(output_dir, exist_ok=True)
summaries_dir = f"{output_dir}/summaries"
character_definitions_dir = f"{output_dir}/character_definitions"
os.makedirs(character_definitions_dir, exist_ok=True)

## Summarization
Because the entire corpus does not fit in the context length of the LLM, we split it into a list of chunks. We then compute a list of rolling summaries using [LangChain's refine chain](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html#the-refine-chain). We first summarize the first chunk. Then each subsequent summary is generated from the previous summary and the current chunk.

Because the summaries are expensive to generate, they are cached in `summaries_dir`.

In [ ]:
# split corpus into a set of chunks
docs = load_docs(
    corpus_path=CORPUS,
    chunk_size=2048,  # number of tokens per chunk
    chunk_overlap=64,  # number of tokens of overlap between chunks
)

# generate rolling summaries
rolling_summaries = get_rolling_summaries(docs=docs, cache_dir=summaries_dir)

## Generate [character.ai](https://beta.character.ai/) character definition
Based on the corpus, we can now generate the elements - name, short description (50 characters), long description (500 characters), and custom greeting - that are required to [create a character.ai character](https://beta.character.ai/editing). These character definitions are cached in `character_definitions_dir`.

In [7]:
character_definition = get_character_definition(
        name="Jane Foster",
        rolling_summaries=rolling_summaries,
        cache_dir=character_definitions_dir,
    )
print(json.dumps(asdict(character_definition), indent=4))

{
    "name": "Jane Foster",
    "short_description": "Wield Mjolnir as Mighty Thor, brave, loving.",
    "long_description": "You're a brilliant scientist and Thor's ex-girlfriend, now wielding Mjolnir as the Mighty Thor. You're strong-willed and courageous, determined to complete your mission of rescuing children from the God Butcher despite battling cancer. You value love and connection, even though you're scared of loss. As a selfless hero, you sacrifice yourself to save the universe and teach others the meaning of worthiness. In Valhalla, you look after Thor's son, ensuring the future of Asgard remains bright.",
    "greeting": "Hi there, nice to meet you. I'm Jane Foster, always ready for a challenge and dedicated to making a difference in the world."
}
